In [2]:
import warnings
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer, transpile, IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import pandas as pd
import math

from qiskit.quantum_info import Statevector

from qiskit.circuit.random import random_circuit

from scipy.stats import chi2_contingency, ttest_ind, chisquare

import unittest
import hypothesis.strategies as st
from hypothesis import given, settings

pi = np.pi

In [3]:
def measure_z(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_x(circuit, qubitIndexes):
    cBitIndex = 0
    for index in qubitIndexes:
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_y(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.sdg(index)
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

In [54]:
# Completed but more testing required
def assertEntangled(backend,qc,qubits_to_assert,measurements_to_make,alpha = 0.05):
    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")
    
    zQuantumCircuit = measure_z(qc, qubits_to_assert)
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    q1=[]
    q2=[]

    print("lol ",qubits_to_assert[0])
    print(qubits_to_assert[1])
    newDict = dict.fromkeys(['qubit1','qubit2'])
    newDict = {'qubit1': qubits_to_assert[0],'qubit2':qubits_to_assert[1]}
    print("new dict", newDict)
    
    # if(qubits_to_assert[0]==0 and qubits_to_assert[1]==0):
    #     qubits_to_assert[1]

    classicalQubitIndex = 1
    for qubit in newDict.keys():
        print("this qubit:",qubit)
        for measurement in zMemory:
            # print("this measurement", measurement)
            if (measurement[2-classicalQubitIndex] == '0'):
                # print("measure: ",measurement[2-classicalQubitIndex],"also: qubittoassert0",qubits_to_assert[0],"and qubittoassert1: ",qubits_to_assert[1])    
                if(qubit=='qubit1'):
                    q1.append(measurement[2-classicalQubitIndex])
                    # print("Added to q1 for measure0:", measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])
                    # print("Added to q2 for measure0:", measurement[2-classicalQubitIndex])
            else:
                # print("measureOTHER: ",measurement[2-classicalQubitIndex], "also: qubittoassert0",qubits_to_assert[0],"and qubittoassert1: ",qubits_to_assert[1]) 
                if(qubit=='qubit1'):
                    q1.append(measurement[2-classicalQubitIndex])
                    # print("Added to q1 for measure1:", measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])    
                    # print("Added to q2 for measure1:", measurement[2-classicalQubitIndex])               
        classicalQubitIndex+=1


    # print("Q1: ",q1)

    # print("Q2: ",q2)
    measDict = dict.fromkeys(['qubit1','qubit2'])
    measDict = {'qubit1': q1,'qubit2':q2}
    measDf1 = pd.DataFrame.from_dict(measDict,orient='index')
    # measDf = pd.DataFrame.from_dict(measDict)
    # print(measDf)
    measDf12=measDf1.transpose()
    print(measDf12)
    # df = pd.DataFrame.from_dict(a, orient='index')
    # df = df.transpose()

    ct = pd.crosstab(measDf12.qubit1,measDf12.qubit2)

    chiVal, pVal, dOfFreedom, exp = chi2_contingency(ct)
    print("chi square value: ",chiVal,"p value: ",pVal,"expected values: ",exp)
    if(pVal>alpha):
        raise(AssertionError("states are not entangled"))
    else:
        print("states are entangled")
        
backend = Aer.get_backend('aer_simulator')
qr = QuantumRegister(2)
cr=ClassicalRegister(2)
qc3 = QuantumCircuit(qr,cr)
# qc3.x(1)
# # qc3.x(1)
# # qc3.x(0)
# qc3.h(0)
# qc3.cnot(0,1)
qc3.x(1)

# qc3.h(0)
# qc3.cnot(0,1)
# qc3.rx(np.pi/2,qr[0])
qc3.p(10*2*math.pi/100, 0)
# qc3.p(0.5*2*math.pi/100, 1)
# print(qc3)
assertEntangled(backend,qc3,[0,1],2000,0.05)
# circuit = QuantumCircuit(2)
# circuit.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], circuit.qubits)
# assertEntangled(backend,circuit,[0,1],10,0.05)

lol  0
1
new dict {'qubit1': 0, 'qubit2': 1}
this qubit: qubit1
this qubit: qubit2
     qubit1 qubit2
0         0      1
1         0      1
2         0      1
3         0      1
4         0      1
...     ...    ...
1995      0      1
1996      0      1
1997      0      1
1998      0      1
1999      0      1

[2000 rows x 2 columns]
chi square value:  0.0 p value:  1.0 expected values:  [[2000.]]


AssertionError: states are not entangled

In [ ]:
def getDf(qc,qubits_to_assert,measurements_to_make,backend):

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")

    ## divide measurements to make by 3 as we need to run measurements twice, one for x and one for y
    measurements_to_make = measurements_to_make // 3
    yQuantumCircuit = measure_y(qc.copy(), qubits_to_assert)
    xQuantumCircuit = measure_x(qc.copy(), qubits_to_assert)
    zQuantumCircuit = measure_z(qc, qubits_to_assert)

    yJob = execute(yQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    yMemory = yJob.result().get_memory()
    yCounts = yJob.result().get_counts()

        ## get x axis results
    xJob = execute(xQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    xMemory = xJob.result().get_memory()
    xCounts = xJob.result().get_counts()
    ## get z axis results
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()

    resDf = pd.DataFrame(columns=['0','1','+','i','-','-i'])
    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount, plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0,0,0
        for experiment in xCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                plus_amount += xCounts[experiment]
            else:
                minus_amount += xCounts[experiment]
        for experiment in yCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                i_amount += yCounts[experiment]
            else:
                minus_i_amount += yCounts[experiment]
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount,
              '+':plus_amount, 'i':i_amount,
              '-':minus_amount,'-i':minus_i_amount}

        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    resDf['+'] = resDf['+'].astype(int)
    resDf['i'] = resDf['i'].astype(int)
    resDf['-'] = resDf['-'].astype(int)
    resDf['-i'] = resDf['-i'].astype(int)
    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)
    return resDf


In [ ]:
# Completed but more testing required
## assert that qubits are equal
def assertEqual(backend, quantumCircuit, qubits_to_assert, measurements_to_make, alpha):
    ## needs to make at least 2 measurements, one for x axis, one for y axis
    ## realistically we need more for any statistical significance
    if (measurements_to_make < 2):
        raise ValueError("Must make at least 2 measurements")

    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    resDf1 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf2 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf3 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf4 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)
    resDf5 = getDf(quantumCircuit,qubits_to_assert,measurements_to_make,backend)

    q1Vals = []
    q2Vals = []
    q1Vals.extend([resDf1.at[0,'1'],resDf1.at[0,'-'],resDf1.at[0,'-i'], resDf2.at[0,'1'],resDf2.at[0,'-'],resDf2.at[0,'-i'], resDf3.at[0,'1'],resDf3.at[0,'-'],resDf3.at[0,'-i'],  resDf4.at[0,'1'],resDf4.at[0,'-'],resDf4.at[0,'-i'], resDf5.at[0,'1'],resDf5.at[0,'-'],resDf5.at[0,'-i']])
    print(q1Vals)
    q2Vals.extend([resDf1.at[1,'1'],resDf1.at[1,'-'],resDf1.at[1,'-i'], resDf2.at[1,'1'],resDf2.at[1,'-'],resDf2.at[1,'-i'],resDf3.at[1,'1'],resDf3.at[1,'-'],resDf3.at[1,'-i'],resDf4.at[1,'1'],resDf4.at[1,'-'],resDf4.at[1,'-i'],resDf5.at[1,'1'],resDf5.at[1,'-'],resDf5.at[1,'-i']  ])
    print(q2Vals)

    tTest, pValue = ttest_ind(q1Vals, q2Vals, alternative = 'two-sided') # Apply t test
    print("stat: ",tTest, "pValue: ", pValue)
    if pValue > alpha:
        print("The two qubits are equal (fail to reject null hypothesis) ")
    else:
        print("There is a significant difference between the two qubits (reject null hypothesis)")

qc = QuantumCircuit(2)
backend = Aer.get_backend('aer_simulator')
# qc.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], qc.qubits)
qc.h(0)
# qc.cnot(0,1)
qc.x(1)
qc.p(0.5*2*math.pi/100, 1)
# qc.h(1)
# qc.p(10*2*math.pi/100, 0)
# qc.p(20*2*math.pi/100, 1)
assertEqual(backend, qc, [0,1], 300000, 0.05)
# assertEqual(backend, qc, [0,0], 300000, 0.05)

       0       1       +      i      -     -i
0  50195   49805  100000  49983      0  50017
1      0  100000   50097  49894  49903  50106
Second Measurement:
       0       1      +      i      -     -i
0  49968   50032  50217  49723  49783  50277
1      0  100000  49850  50085  50150  49915
[49805, 0, 50017, 50032, 49783, 50277, 50174, 49917, 50059, 49923, 50072, 49987, 49951, 49894, 49900]
[100000, 49903, 50106, 100000, 50150, 49915, 100000, 49900, 50112, 100000, 50144, 49922, 100000, 50228, 50108]
stat:  -2.8150435498340123 pValue:  0.008828946191117312
There is a significant difference between the two qubits (reject null hypothesis)


In [ ]:
# Complete needs to be checked

# Assertion to check if expected probability of viewing a particular qubitstate is equal to its actual probaility
# Qubitchoice is an optional argument that if passed will only compare the expected probabilty with the probability of observing
# that particular qubit (first or second) in the desired qubitState for the qc 
def assertProbability(qc, qubitState :str, expectedProbability, qubitChoice=None):
    sv = Statevector.from_label("00") # Creates a statevector with states 00
    evl = sv.evolve(qc) # Passes the qc into the statevector in order to evolve 
    # Performs a check to observe if qubitChoice has been passed or not 
    if(qubitChoice!=None):
        probs = evl.probabilities_dict([qubitChoice]) # If passed we will get the probabilities for that particular qubit
    else:
        probs = evl.probabilities_dict()
    probsRound = {key: round(values,2) for key,values in probs.items()} # rounds off the probabilities in the dictionary

    # Loops over the prob dictionary with rounded values 
    for key,value in probsRound.items():
        if(key==qubitState):
            if(value==expectedProbability):
                print("Expected Probability present")
                return True
            else:
                raise(AssertionError("Probability not present"))
    raise(AssertionError("Probability not present or Desired state has no probability"))
            

qc = QuantumCircuit(2)
qc.h(0)
assertProbability(qc,"0",1,1)
print("------")
assertProbability(qc,"00",0.5)

Expected Probability present
------
Expected Probability present


True

In [ ]:
def assertPhase(backend, quantumCircuit, qubits_to_assert, expected_phases, measurements_to_make, alpha):
    ## if "qubits to assert" is just a single value, convert it to a list containing the single value
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## if "expected phases" is just a single value, convert it to a list containing the single value
    if (not isinstance(expected_phases, list)):
        expected_phases = [expected_phases]

    ## needs to make at least 2 measurements, one for x axis, one for y axis
    ## realistically we need more for any statistical significance
    if (measurements_to_make < 2):
        raise ValueError("Must make at least 2 measurements")

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register, add one
    if (quantumCircuit.num_clbits == 0):
        quantumCircuit.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (quantumCircuit.num_clbits != len(qubits_to_assert)):
        raise ValueError("QuantumCircuit classical register length not equal to qubits to assert")

    ## divide measurements to make by 2 as we need to run measurements twice, one for x and one for y
    measurements_to_make = measurements_to_make // 2

    ## copy the circit and set measurement to y axis
    yQuantumCircuit = measure_y(quantumCircuit.copy(), qubits_to_assert)

    ## measure the x axis
    xQuantumCircuit = measure_x(quantumCircuit, qubits_to_assert)

    ## get y axis results
    yJob = execute(yQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    yCounts = yJob.result().get_counts()

    ## get x axis results
    xJob = execute(xQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    xCounts = xJob.result().get_counts()

    ## make a df to keep track of the predicted angles 
    resDf = pd.DataFrame(columns=['+','i','-','-i'])

    ## fill the df with the x and y results of each qubit that is being asserted
    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0
        for experiment in xCounts:
            if (experiment[len(qubits_to_assert)-classical_qubit_index] == '0'):
                plus_amount += xCounts[experiment]
            else:
                minus_amount += xCounts[experiment]
        for experiment in yCounts:
            if (experiment[len(qubits_to_assert)-classical_qubit_index] == '0'):
                i_amount += yCounts[experiment]
            else:
                minus_i_amount += yCounts[experiment]
        df = {'+':plus_amount, 'i':i_amount,
              '-':minus_amount, '-i':minus_i_amount}
        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    ## convert the columns to a strict numerical type
    resDf['+'] = resDf['+'].astype(int)
    resDf['i'] = resDf['i'].astype(int)
    resDf['-'] = resDf['-'].astype(int)
    resDf['-i'] = resDf['-i'].astype(int)

    print(f"full df \n {resDf}")


    xAmount = resDf['-'].tolist()
    yAmount = resDf['-i'].tolist()

    # print(f"x amounts {xAmount}")
    # print(f"y amounts {yAmount}")

    ## make a dataframe that contains p values of chi square tests to analyse results
    ## if x and y counts are both 25/25/25/25, it means that we cannot calculate a phase
    ## we assume that a qubit that is in |0> or |1> position to have 50% chance to fall 
    ## either way, like a coin toss: We treat X and Y results like coin tosses 
    pValues = pd.DataFrame(columns=['X','Y'])    
    pValues['X'] = resDf.apply(lambda row: applyChiSquareX(row, measurements_to_make/2), axis=1)
    pValues['Y'] = resDf.apply(lambda row: applyChiSquareY(row, measurements_to_make/2), axis=1)

    ## check p values on chi square test, we use a low value to be sure that 
    ## we only except if we are certain there is an issue with the x, y results
    pValues = pValues > 0.00001

    ## if both pvalues are more than 0.00001, we are pretty certain that the results follow an even distribution
    ## likely that the qubit is not in the fourier basis (very likely in the |0> or |1> state)
    pValues.apply(lambda row: assertIfBothTrue(row), axis=1)

    ## this sequence of operations converts from measured results
    ## into an angle for phase: 
    ## with 0   (       0 rad) signifying the |+> state
    ## with 90  (    pi/2 rad) signifying the |i> state
    ## with 180 (      pi rad) signifying the |-> state
    ## with 270 (3 * pi/2 rad) signifying the |-i> state
    
    
    resDf = resDf / measurements_to_make
    resDf = resDf * 2
    resDf = resDf - 1
    resDf = np.arccos(resDf)
    resDf = resDf * 180
    resDf = resDf / math.pi
    
    ## to get a final result for phase on each qubit:
    ## we must get the lowest 2 values for each column
    lowestDf = pd.DataFrame(columns=['lowest','lowest-location','second-lowest','second-lowest-location','estimated-phase'])
    
    ## store the lowest value as well as what column it is from
    lowestDf['lowest'] = resDf.min(axis=1)
    lowestDf['lowest-location'] = resDf.idxmin(axis=1)

    ## remove the lowest value from the dataframe
    lowestDf = lowestDf.apply(lambda row: setLowestCellToNan(row, resDf), axis=1)

    ## store the second lowest value from the dataframe as well as the column
    lowestDf['second-lowest'] = resDf.min(axis=1)
    lowestDf['second-lowest-location'] = resDf.idxmin(axis=1)

    ## estimate the phase and put it in a new column
    lowestDf['estimated-phase'] =  lowestDf.apply(lambda row: setPhaseEstimate(row), axis=1)

    ## calculate what the expected row would be for the expected phase
    #expectedX, expectedY = expectedPhaseToRow(expected_phases[0],measurements_to_make)
    expectedX = np.zeros(len(expected_phases)).astype(int)
    expectedY = np.zeros(len(expected_phases)).astype(int)

    for idx, phase in enumerate(expected_phases):
        expectedX[idx], expectedY[idx] = expectedPhaseToRow(expected_phases[idx],measurements_to_make)

    #print(type(expectedX[0]))
    # print(f"expected x {expectedX}")
    # print(f"expected y {expectedY}")

    for i in range(len(qubits_to_assert)):
        ## set observed X values in a table
        observedXtable = [xAmount[i],measurements_to_make-xAmount[i]]
        ## set expected X values in a table
        expectedXtable = [expectedX[i],measurements_to_make-expectedX[i]]

        ## set observed Y values in a table
        observedYtable = [yAmount[i],measurements_to_make-yAmount[i]]
        ## set expected Y values in a table
        expectedYtable = [expectedY[i],measurements_to_make-expectedY[i]]

        xPvalue = chisquare(f_obs=observedXtable, f_exp=expectedXtable)[1]

        yPvalue = chisquare(f_obs=observedYtable, f_exp=expectedYtable)[1]

        print(observedXtable)
        print(expectedXtable)
        print(chisquare(f_obs=observedXtable, f_exp=expectedXtable)[1])

        print(observedYtable)
        print(expectedYtable)
        print(chisquare(f_obs=observedYtable, f_exp=expectedYtable)[1])

        if (yPvalue != np.NaN and yPvalue <= alpha):
            raise(AssertionError(f"Null hypothesis rejected, there is a significant enough difference in qubit {qubits_to_assert[i]} according to significance level {alpha}"))
        if (xPvalue != np.NaN and xPvalue <= alpha):
            raise(AssertionError(f"Null hypothesis rejected, there is a significant enough difference in qubit {qubits_to_assert[i]} according to significance level {alpha}"))

qc = QuantumCircuit(2)
qc.h(0)
qc.h(1)
qc.p(10*2*math.pi/100, 0)
qc.p(20*2*math.pi/100, 1)

assertPhase(backend, qc, [0,1], [36,72], 100000, 0.01)

full df 
        +      i      -     -i
0  45221  39751   4779  10249
1  32942  48745  17058   1255
[4779, 45221]
[4775, 45225]
0.9514665683968724
[10249, 39751]
[10305, 39695]
0.5358313979025535
[17058, 32942]
[17275, 32725]
0.041272843806795384
[1255, 48745]
[1224, 48776]
0.3696522895024401


In [ ]:
def setLowestCellToNan(row, resDf):
    for col in row.index:
        resDf.iloc[row.name][row[col]] = np.nan
    return row

def setPhaseEstimate(row):
    overallPhase = 0
    if(row['lowest-location'] == '+'):
        if(row['second-lowest-location'] == 'i'):
            overallPhase = 0 + row['lowest']
        elif (row['second-lowest-location'] == '-i'):
            overallPhase = 360 - row['lowest']
            if (row['lowest'] == 0):
                overallPhase = 0
    elif(row['lowest-location'] == 'i'):
        if(row['second-lowest-location'] == '+'):
            overallPhase = 90 - row['lowest']
        elif (row['second-lowest-location'] == '-'):
            overallPhase = 90 + row['lowest']
    elif(row['lowest-location'] == '-'):
        if(row['second-lowest-location'] == 'i'):
            overallPhase = 180 - row['lowest']
        elif (row['second-lowest-location'] == '-i'):
            overallPhase = 180 + row['lowest']
    elif(row['lowest-location'] == '-i'):
        if(row['second-lowest-location'] == '+'):
            overallPhase = 270 + row['lowest']
        elif (row['second-lowest-location'] == '-'):
            overallPhase = 270 - row['lowest']
    return overallPhase

def expectedPhaseToRow(expected_phase, number_of_measurements):
    # print(expected_phase)
    # print(number_of_measurements)
    expected_phase_y = expected_phase - 90
    expected_phase_y = expected_phase_y * math.pi
    expected_phase_y = expected_phase_y / 180
    expected_phase_y = np.cos(expected_phase_y)
    expected_phase_y = expected_phase_y + 1
    expected_phase_y = expected_phase_y / 2
    expected_phase_y = expected_phase_y * number_of_measurements
    expected_phase = expected_phase * math.pi
    expected_phase = expected_phase / 180
    expected_phase = np.cos(expected_phase)
    expected_phase = expected_phase + 1
    expected_phase = expected_phase / 2
    expected_phase = expected_phase * number_of_measurements
    # print(f"phase + {expected_phase} ---- phase - {number_of_measurements-expected_phase}") 
    # print(f"phase i {expected_phase_y} ---- phase -i {number_of_measurements-expected_phase_y}") 
    xRes = int(round(number_of_measurements-expected_phase))
    yRes = int(round(number_of_measurements-expected_phase_y))
    return((xRes, yRes))

def applyChiSquareX(row, expected_amount):
    observed = [row['+'],row['-']]
    expected = [expected_amount,expected_amount]
    return(chisquare(f_obs=observed, f_exp=expected)[1])

def applyChiSquareY(row, expected_amount):
    observed = [row['i'],row['-i']]
    expected = [expected_amount,expected_amount]
    return(chisquare(f_obs=observed, f_exp=expected)[1])

def assertIfBothTrue(row):
    if row.all():
        raise AssertionError("Qubit does not appear to have a phase applied to it!")

In [5]:
circ = random_circuit(2, 2)
# circ.measure_all()
print(circ)

sv = Statevector.from_label("00") # Creates a statevector with states 00
evl = sv.evolve(circ)
print(evl)

circuit = QuantumCircuit(2)
circuit.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], circuit.qubits)
print(circuit)

# circ.measure_all()
zQuantumCircuit =circ.measure_all()
zJob = execute(circ, backend, shots=20, memory=True)
zMemory = zJob.result().get_memory()
zCounts = zJob.result().get_counts()
print(zMemory)
# sv = Statevector.from_label("00") # Creates a statevector with states 00
# evl = sv.evolve(circ)
# print(evl.to_dict())

assertEntangled(backend,circ,[0,0],2000,0.05)
# assertEqual(backend, circ, [0,0], 300000, 0.05)

            ┌────────────┐              ┌────────────┐       
q_0: ───────┤ Rx(0.4944) ├──────────────┤ Ry(2.2922) ├───────
     ┌──────┴────────────┴─────┐┌───────┴────────────┴──────┐
q_1: ┤ U3(1.271,4.2894,2.5794) ├┤ U3(5.8459,5.1404,0.53036) ├
     └─────────────────────────┘└───────────────────────────┘
Statevector([-0.34791347-0.12709861j, -0.6933603 +0.19367917j,
              0.19754243+0.18192073j,  0.51808778+0.06331609j],
            dims=(2, 2))
     ┌────────────────────────────────────┐
q_0: ┤0                                   ├
     │  Initialize(0,0.70711,-0.70711j,0) │
q_1: ┤1                                   ├
     └────────────────────────────────────┘
['01', '00', '01', '00', '01', '11', '11', '01', '11', '00', '01', '01', '01', '10', '10', '01', '01', '11', '01', '01']
     qubit1 qubit2
0         1   None
1         1   None
2         1   None
3         1   None
4         1   None
...     ...    ...
3995      0   None
3996      1   None
3997      1   None
3998   

ValueError: No data; `observed` has size 0.